## データ前処理

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

In [3]:
dataset=pd.read_csv('sample_data.csv')
dataset.head()

,物件名,カテゴリー,住所,最寄駅0,最寄駅1,最寄駅2,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積
0,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,東京都台東区清川２,東京メトロ日比谷線/南千住駅 歩8分,ＪＲ常磐線/南千住駅 歩10分,つくばエクスプレス/南千住駅 歩10分,新築,11階建,2階,8万円,10000円,-,-,1K,25.23m2
1,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,東京都台東区清川２,東京メトロ日比谷線/南千住駅 歩8分,ＪＲ常磐線/南千住駅 歩10分,つくばエクスプレス/南千住駅 歩10分,新築,11階建,2階,8万円,10000円,-,-,1K,25.23m2
2,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,東京都台東区清川２,東京メトロ日比谷線/南千住駅 歩8分,ＪＲ常磐線/南千住駅 歩10分,つくばエクスプレス/南千住駅 歩10分,新築,11階建,11階,8.45万円,10000円,-,-,1K,25.23m2
3,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,東京都台東区清川２,東京メトロ日比谷線/南千住駅 歩8分,ＪＲ常磐線/南千住駅 歩10分,つくばエクスプレス/南千住駅 歩10分,新築,11階建,11階,8.45万円,10000円,-,-,1K,25.23m2
4,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,東京都台東区清川２,東京メトロ日比谷線/南千住駅 歩8分,ＪＲ常磐線/南千住駅 歩10分,つくばエクスプレス/南千住駅 歩10分,新築,11階建,8階,13.2万円,10000円,-,13.2万円,2LDK,40.25m2


* カテゴリーはカテゴリカル変数に変換
* 住所は23区と都市名 
* 最寄駅は最も近い駅名とその時間 
* 築年数はint型に変換 
* 構造、階数、賃料、管理費、敷金、礼金はint型に変換 
* 間取りはカテゴリカル変数に変換 
* 面積は単位なしのfloat型に変換


In [169]:
def preprocessing(df):
    df_new=df.copy()
    
    wards=[]
    citys=[]
    lines=[]
    stations=[]
    distances=[]
    results=[]
    ages=[]
    builds=[]
    Floors=[]
    prices=[]
    kanrihis=[]
    sikikins=[]
    reikins=[]
    areas=[]
    
    for i in range(0,len(df_new)):
        ###住所を'23区'、'都市名'に分ける
        pattern='.*区'
        repatter = re.compile(pattern)
        result = repatter.match(df['住所'][i])        
        wards.append(result.group()[3:])
        citys.append(df['住所'][i].split('区')[1])
    
        ###最寄駅は最も近い駅名、路線と時間を採用する
        nearest_station=df['最寄駅0'][i]
        lines.append(nearest_station.split('/')[0])
        stations.append(nearest_station.split('/')[1].split(' ')[0])
        distances.append(int(nearest_station.split(' ')[1][1]))
        
        ###築年数はint型に変換
        age=dataset['築年数']
        if age[i][0]=='新':
            ages.append(0)
        else:   
            ages.append(int(age[i].split('年')[0][1:]))
        
        ###構造
        build=dataset['構造']
        builds.append(int(build[i].split('階')[0]))
        
        ###部屋の階数
        floor=dataset['階数']
        if len(floor[i].split('階')[0])>2 and floor[i].split('階')[0][1]=='-':
            Floors.append(random.randint(1,builds[i]))
        else:
            Floors.append(int(floor[i].split('階')[0]))
        
        ###賃料
        price=dataset['賃料']
        prices.append(float(price[i].split('万')[0])*10000)
        
        ###管理費
        kanrihi=dataset['管理費']
        if kanrihi[i]!='-':
            kanrihis.append(int(kanrihi[i].split('円')[0]))
        else:
            kanrihis.append(float(0))
        
        ###敷金
        sikikin=dataset['敷金']
        if sikikin[i]!='-':
            sikikins.append(float(sikikin[i].split('万')[0]))
        else:
            sikikins.append(float(0))
                
        ###礼金
        reikin=dataset['礼金']
        if reikin[i]!='-':
            reikins.append(float(reikin[i].split('万')[0]))
        else:
            reikins.append(float(0))
        
        ###面積
        area=dataset['面積']
        areas.append(float(area[i].split('m')[0]))
        
        
    ###変換したカラムの追加
    df_new['23区']=wards
    df_new['都市名']=citys
    df_new['路線']=lines
    df_new['最寄駅']=stations
    df_new['徒歩']=distances
    df_new['築年数']=ages
    df_new['構造']=builds
    df_new['階数']=Floors
    df_new['賃料']=prices
    df_new['管理費']=kanrihis
    df_new['敷金']=sikikins
    df_new['礼金']=reikins
    df_new['面積']=areas
    
#     ###カテゴリカル変数に変換
#     df_new['カテゴリー']=df['カテゴリー'].astype('category') 
#     df_new['間取り']=df_new['間取り'].astype('category')
#     df_new['23区']=df_new['23区'].astype('category')
#     df_new['都市名']=df_new['都市名'].astype('category')
#     df_new['路線']=df_new['路線'].astype('category')
#     df_new['最寄駅']=df_new['最寄駅'].astype('category')

    ###不要なカラムを削除
    df_new=df_new.drop('住所',axis=1)
    df_new=df_new.drop(['最寄駅0', '最寄駅1', '最寄駅2'],axis=1)    
    
    return df_new

df_new=preprocessing(dataset)
df_new.head()

,物件名,カテゴリー,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,面積,23区,都市名,路線,最寄駅,徒歩
0,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,0,11,2,80000.0,10000.0,0.0,0.0,1K,25.23,台東区,清川２,東京メトロ日比谷線,南千住駅,8
1,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,0,11,2,80000.0,10000.0,0.0,0.0,1K,25.23,台東区,清川２,東京メトロ日比谷線,南千住駅,8
2,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,0,11,11,84500.0,10000.0,0.0,0.0,1K,25.23,台東区,清川２,東京メトロ日比谷線,南千住駅,8
3,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,0,11,11,84500.0,10000.0,0.0,0.0,1K,25.23,台東区,清川２,東京メトロ日比谷線,南千住駅,8
4,東京メトロ日比谷線 南千住駅 11階建 新築,賃貸マンション,0,11,8,132000.0,10000.0,0.0,13.2,2LDK,40.25,台東区,清川２,東京メトロ日比谷線,南千住駅,8


In [170]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755 entries, 0 to 1754
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   物件名     1755 non-null   object 
 1   カテゴリー   1755 non-null   object 
 2   築年数     1755 non-null   int64  
 3   構造      1755 non-null   int64  
 4   階数      1755 non-null   int64  
 5   賃料      1755 non-null   float64
 6   管理費     1755 non-null   float64
 7   敷金      1755 non-null   float64
 8   礼金      1755 non-null   float64
 9   間取り     1755 non-null   object 
 10  面積      1755 non-null   float64
 11  23区     1755 non-null   object 
 12  都市名     1755 non-null   object 
 13  路線      1755 non-null   object 
 14  最寄駅     1755 non-null   object 
 15  徒歩      1755 non-null   int64  
dtypes: float64(5), int64(4), object(7)
memory usage: 219.5+ KB


In [171]:
df_new.to_csv('preprocessed_data.csv', index=None)